In [ ]:
import pyemu
import conda_scripts.load_datasets as lsd
import os
import pandas as pd
import numpy as np
import conda_scripts
import forward_run
import matplotlib.pyplot as plt
import conda_scripts.plot_help as ph
from conda_scripts import sv_budget
import matplotlib as mpl
import numpy as np

import geopandas as gpd

In [ ]:
workspace = lsd.model_info.get_mod('son', True)
workspace

In [ ]:
main = 'postprocess'
fold =  'pestaws_v1'

out_folder = os.path.join(main, fold)
if not os.path.exists(out_folder):
    os.makedirs(out_folder, exist_ok =True)

In [ ]:
ml = conda_scripts.sv_budget.load_sv_model.get_model(workspace)

In [ ]:
m_d = f_d = "C:\GSP\sv\model\pest_inputs\PEST_PP\pestaws"

In [ ]:
pst = pyemu.Pst(os.path.join(f_d,"pest_ies.pst"))

In [ ]:
os.listdir(m_d)

In [ ]:
pst.parrep(os.path.join(m_d, 'pest_ies.3.par.csv'), noptmax = 3)

In [ ]:
pst.write(os.path.join(r"C:\GSP\sv\model\SV_mod_V2_owhm2\master", 'pest_ies.pst'), version = 2)

In [ ]:
pst.res.head()

In [ ]:
pst.res.query("weight>0")

In [ ]:
pst.res.query("weight>0").groupby('group').plot.scatter('measured', 'modelled',grid = True,legend =True)

In [ ]:
# the value of the objective function
pst.phi

In [ ]:
pst.plot(kind="1to1",verbose = True) # requires Pst.res
plt.savefig(os.path.join(out_folder, '1to1.png'))
pst.plot(kind="prior")
plt.savefig(os.path.join(out_folder, 'prior.png'))
pst.plot(kind="phi_pie")
plt.savefig(os.path.join(out_folder, 'pie.png'))

In [ ]:
pst.parameter_data

In [ ]:
pst.bounds_report()

In [ ]:
pst.observation_data

In [ ]:
pst.res.group.unique()

In [ ]:
pst.res.query("name.str.contains('agua_cal') and  name.str.contains('raw') and not measured<0 ").plot.scatter('measured','modelled')

In [ ]:
pst.get_res_stats()

In [ ]:
fig, axes = plt.subplots(4, 2, sharey=True,sharex = True, figsize=(10,3.5))


phi = pd.read_csv(os.path.join(f_d,"pest_ies.phi.actual.csv"),index_col=0)
noi = pd.read_csv(os.path.join(f_d,"pest_ies.phi.meas.csv"),index_col=0)
# phi.index = phi.total_runs
# phi.index = phi.total_runs
for it, row in phi.iterrows():
    ax = axes[it,0]
    phi.iloc[it,6:].apply(np.log10).plot.hist(legend=False,lw=0.5,color='k', ax=ax, bins = np.arange(9,11,.1), label = 'asdf')

        # Add axvline for each mean value with different colors
    colors = ['purple', 'orange', 'cyan']
    

    ax.axvline(x=np.log10(phi.loc[it,'mean']), color=colors[0], linestyle='-')
    ax.axvline(x=np.log10(phi.loc[it,'min']), color=colors[1], linestyle='--')
    ax.axvline(x=np.log10(phi.loc[it,'max']), color=colors[1], linestyle='--')
    ax.fill_betweenx([0,150], np.log10(phi.loc[it,'min']),
                     np.log10(phi.loc[it,'max']), color='grey', alpha=0.3)
    ax.set_ylabel(r'count')
    if it==0:
        ax.set_title(r'Actual $\Phi$')


    
    # right
    ax = axes[it,1]
    noi.iloc[it,6:].apply(np.log10).plot.hist(legend=False,lw=0.2,color='r', ax=ax, bins = np.arange(9,11,.1))
    ax.axvline(x=np.log10(noi.loc[it,'mean']), color=colors[0], linestyle='-', label = "Mean")
    ax.axvline(x=np.log10(noi.loc[it,'min']), color=colors[1], linestyle='--', label = 'min/max')
    ax.axvline(x=np.log10(noi.loc[it,'max']), color=colors[1], linestyle='--')
    ax.fill_betweenx([0,150], np.log10(noi.loc[it,'min']),
                 np.log10(noi.loc[it,'max']), color='grey', alpha=0.3)
    
    if it==0:
        ax.set_title(r'Measured+Noise $\Phi$')
        fig.tight_layout()
        # ax.set_ylim([None, 11])
        ax.legend()

plt.savefig(os.path.join(out_folder, 'phi_reduction.png'))

In [ ]:
plt.figure()
phi.iloc[-1,6:].hist()
plt.title(r'Final $\Phi$ Distribution');

PESTPP-IES has conveniently kept track of all our observation data, measurement noise and the model outputs from each realization at each iteration. This now allows us to go back and look at all this information in detail should we wish to do so. We are interested in looking at (1) how model outputs compare to measured data+noise and (2) the distribution of model outputs for forecast observations.

Since PESTPP-IES evaluates a prior parameter ensemble, we can use the model outputs from that iteration (iteration zero) as a sample of the prior. We treat the model outputs from the ensemble for the final (best?) iteration as a sample of the posterior. Let's read in the files which PESTPP-IES recorded:

In [ ]:
pr_oe = pyemu.ObservationEnsemble.from_csv(pst=pst,filename=os.path.join(m_d,"pest_ies.0.obs.csv"))
pt_oe = pyemu.ObservationEnsemble.from_csv(pst=pst,filename=os.path.join(m_d,"pest_ies.{0}.obs.csv".format(pst.control_data.noptmax)))
noise = pyemu.ObservationEnsemble.from_csv(pst=pst,filename=os.path.join(m_d,"pest_ies.obs+noise.csv"))

In [ ]:
pt_oe

In [ ]:
phi_per = pd.read_csv(os.path.join(m_d,"pest_ies.phi.meas.csv"), index_col = 0)
phi_per

In [ ]:
good = (phi_per.loc[[3],'0':]< (phi_per.at[3,'mean'] + phi_per.at[3,'standard_deviation'] ))
good = good.T
good  = good.loc[good.values]
good

Make a quick comparison of Phi between the prior and posterior. Nice improvement overall!

In [ ]:
fig,ax = plt.subplots(1,1)
pr_oe.phi_vector.apply(np.log10).hist(ax=ax,fc="0.5",ec="green",alpha=1.,density=False, bins = np.arange(9,11,.05), label = 'Prior')
pt_oe.phi_vector.apply(np.log10).hist(ax=ax,fc="None",ec="b",hatch = '/////',alpha=.51,density=False, bins = np.arange(9,11,.05), label = "Posterior")
_ = ax.set_xlabel("$log_{10}\\phi$")
ax.legend()
plt.savefig(os.path.join(out_folder, 'phisimple.png'))

Finally, what we really want to see. Let's plot timeseries of simulated versus measured observation values. We are going to do this many times in this notebook, so let's make a function:

In [ ]:
pst.observation_data.head()

In [ ]:
def plot_tseries_ensembles(pr_oe, pt_oe, noise, onames=['sfr']):
    pst.try_parse_name_metadata()
    # get the observation data from the control file and select 
    obs = pst.observation_data.copy()
    # onames provided in oname argument
    obs = obs.loc[obs.oname.apply(lambda x: x in onames)]
    # only non-zero observations
    obs = obs.loc[obs.obgnme.apply(lambda x: x in pst.nnz_obs_groups),:]
    # make a plot
    ogs = obs.obgnme.unique()
    fig,axes = plt.subplots(len(ogs),1,figsize=(10,2*len(ogs)))
    print(ogs)

    ogs.sort()
    # for each observation group (i.e. timeseries)
    for ax,og in zip(axes,ogs):
        # get values for x axis
        oobs = obs.loc[obs.obgnme==og,:].copy()
        oobs.loc[:,"time"] = oobs.time.astype(float)
        oobs.sort_values(by="time",inplace=True)
        tvals = oobs.time.values
        onames = oobs.obsnme.values


        # plot measured+noise 
        oobs = oobs.loc[oobs.weight>0,:]
        tvals = oobs.time.values
        onames = oobs.obsnme.values
        [ax.plot(tvals,noise.loc[i,onames].values,"grey",lw=0.1,alpha=0.5) for i in noise.index]
        
        
        # plot prior
        [ax.plot(tvals,pr_oe.loc[i,onames].values,"r",lw=0.1,alpha=0.5) for i in pr_oe.index]

        # plot posterior
        [ax.plot(tvals,pt_oe.loc[i,onames].values,"b",lw=0.1,alpha=0.5) for i in pt_oe.index]

        #plot observed
        ax.plot(oobs.time,oobs.obsval,"k-",lw=2, alpha = 1.)
        
        ax.set_title(og,loc="left")
        ax.set_ylim([0,9])
        ax.grid(True)

        # Define the labels and corresponding colors
        labels = ['Posterior', 'Meas. + Noise', 'Prior', 'Observed']
        colors = ['blue', 'grey', 'red', 'black']
        
        
        # Plot dummy lines to create legend entries
        for color, label in zip(colors, labels):
            ax.plot([], [], color=color, label=label)
        
        # Add the legend to the plot
        ax.legend(bbox_to_anchor = (1,1), loc='upper left')
        
    fig.tight_layout()
    return fig

posterior = blue  
noise = grey  
prior = red  
observed = black  

OK, now plot the time series of non-zero observation groups. Let's just plot the timeseries of absolute values of heads and stream gage flow.

In the plots below, red lines are timeseries simulated with the prior parameter ensemble. BLUE lines are model outputs simulated using the posterior parameter ensemble. GREY lines are the ensemble of measurement + noise. OBSERVED in black.

Looks like we are getting an excellent fit. All the blue lines are within the same areas as the red lines. This implies we have achieved a level of fit commensurate with measurement noise. Sounds very positive. What do you think? Success? 

In [ ]:
fig = plot_tseries_ensembles(pr_oe, pt_oe, noise,)

plt.savefig(os.path.join(out_folder, 'sfr_flows.png'))

In [ ]:
pp = pt_oe.copy()

cols = pd.DataFrame.from_records(pp.columns.str.split(':'))
cols = pd.MultiIndex.from_frame(cols, names= ['oname', 'name', 'lst', 'group', 'time'])
cols

pp = pd.DataFrame(index = pp.index, data = pp.values, columns = cols )
pp = pp.T
pp

In [ ]:
flux.index.get_level_values(3).unique()

In [ ]:
flux = pp.query("name=='flux_otype'")

def plot_flux(flux, flux_n = 'sfr_in_date_start', box = False, plot = True):
    sfr = flux.query(f"group=='{flux_n}'").droplevel([0,1,2,3])
    
    
    sfr = sfr.loc[:,good.index]
    sfr = sfr.loc['1970':,:]

    if plot:
        if box == True:
            sfr.T.boxplot(rot = 90)
        else:
            sfr.plot(cmap = 'jet',title = flux_n, legend = False)
        
    return sfr

sfr = plot_flux(flux, box = True )

sfr = sfr.loc[:, (sfr.sum()> np.percentile(sfr.sum(), 25)) &  (sfr.sum()< np.percentile(sfr.sum(), 75)) ]


sfr = plot_flux(flux, 'percent_error_date_start' )
sfr = plot_flux(flux, 'fmp_wells_out_date_start' )


stor_in = plot_flux(flux, 'storage_in_date_start',plot = False )
stor_out = plot_flux(flux, 'storage_out_date_start' ,plot = False)
(stor_out-stor_in).cumsum().div(43560).plot(cmap = 'jet',title = "Cumumlative Storage Change", legend = False, alpha = .25)


stor_in = plot_flux(flux, 'fmp_fnr_out_date_start',plot = False )
stor_out = plot_flux(flux, 'fmp_fnr_in_date_start' ,plot = False)
(stor_in - stor_out-0).div(43560).plot(cmap = 'jet',title = "SFR Exchange", legend = False, alpha = .25)

sfr = plot_flux(flux, 'drt_out_date_start' )
# sfr.plot(cmap = 'jet')

In [ ]:
pp.index.get_level_values(1).unique()

In [ ]:
obs = pst.observation_data.copy()
obs.query("obsnme.str.contains('simulatedequivalent_observationname')")

In [ ]:
hds = pp.query("name=='hds_otype'")
# hds = hds.set_index(hds.index.get_level_values(4), append = True)

In [ ]:
hds = hds.set_index(hds.index.get_level_values(4).str.upper())

hds = hds.loc[:, hds.columns.isin(good.index)]


In [ ]:
import flopy 

In [ ]:
h = flopy.modflow.ModflowHob.load(os.path.join(ml.model_ws, 'SV_HOBS_11_18_2015.HOB'), ml)

In [ ]:
allmea = list()

for r in  ml.HOB.obs_data:
    cur = list()
    if r.multilayer:
        multi = list(r.mlay.keys())
    else:
        multi = [r.layer]
    for k in r.time_series_data:
        # print(r)
        cur.extend(( r.row, r.column, k.totim, k.hobs, k.obsname, r.layer, r.nobs, r.multilayer, multi))

    allmea.extend(cur)


In [ ]:
hobs = pd.DataFrame(np.array(allmea).reshape((-1,9)), columns = ['row' ,'column', 'totim', 'hobs', 'obsname', 'layer', 'nobs','multilayer', 'mlays'])

In [ ]:
dumb = pd.get_dummies(hobs['mlays'].explode()).groupby(level=0).max()
dumb = dumb.rename(columns = lambda x: f"lay{x}")
hobs = hobs.join(dumb)

In [ ]:
hobs.head()

In [ ]:
h = hobs.astype({'column':int, 'row':int, 'totim':float, 'hobs':float, 'layer':int, 'obsname':str, 'nobs':int})

unq = h.drop_duplicates(['row', 'column', 'layer']).loc[:,['row', 'column', 'layer']]
unq.loc[:,'wellnum'] = np.arange(unq.shape[0])

h = pd.merge(h, unq, on = ['row', 'column', 'layer'])

h.loc[:,'date'] = pd.to_datetime('12/1/1969') + pd.to_timedelta([int(x) for x in h.totim], unit = "D")

In [ ]:
h

In [ ]:
hall = pd.merge(h, hds, left_on = 'obsname', right_index = True)

In [ ]:
hall.head(3)

In [ ]:
path = os.path.join(out_folder, 'hydros')
if not os.path.exists(os.path.join(out_folder, 'hydros')):
    os.mkdir(path)

In [ ]:
geom = conda_scripts.arich_functions.get_active_mod_df(ml.modelgrid, ml)

In [ ]:
def make_plot(name,row, col):
    from matplotlib.gridspec import GridSpec
    fig = plt.figure(figsize = (10,6))

    gs1 = GridSpec(1, 2, left=0.05, right=0.48, wspace=0.05,width_ratios=[7, 2],)
    ax = fig.add_subplot(gs1[0, :-1])

    ax.set_title(f'{name}\n\nSVIGFM V2 Simulated Head')  
    ph.baseline(ax,hard = True,yearstart = 1970)

    
    ax.tick_params(labelbottom=True, labelleft=True)
    ax.set_ylabel('Hydraulic Head (feet)')
    ax.grid(True)

    
    ax3 = fig.add_subplot(gs1[-1, -1])
    ax3.tick_params(labelbottom=False, labelleft=False)
    sv_budget.sv_budget.sv_mod_map(simple = True,ax = ax3)
    annotations = [child for child in ax3.get_children() if isinstance(child, mpl.text.Text)]
    annotations[0].remove()
    ax3.legend().remove()
    pt = geom.query(f"row=={row} and col =={col}").geometry.centroid
    print(pt)
    
    pt.plot(ax = ax3,  markersize = 40,marker = '*', color = 'r') 
    return fig, ax



In [ ]:
hall

In [ ]:
c = 0
for n, row in hall.sort_values('nobs', ascending=False).groupby('wellnum'):
    rowind = row['row'].unique()[0]
    column = row['column'].unique()[0]
    
    fig, ax = make_plot('',rowind,column)
    temp = row.set_index('date').loc[:,'0':].sort_index()
    sub = temp.iloc[0,:]
    temp.iloc[1:] = temp.iloc[1:].add(sub)
    # row.set_index('date').loc[:,'0':].sort_index().add(row.set_index('date').loc[:,'0':].sort_index().iloc[0,:])
    
    
    temp.plot(ax = ax, legend = False, alpha = .2)
    row.set_index('date').loc[:,['hobs']].sort_index().plot(ax = ax, legend = False, c = 'k', label = "Observed")
    # print(row.drop_duplicates())
    lays = row.iloc[[0],:].reset_index().at[0,'mlays']
    lays = ', '.join([str(j+1) for j in lays])
    ax.set_title( "".join(row.obsname.unique()[0].split("_")[:-1]) + f", layer {lays}" )
    c = c+1
    ph.yrange_(ax)
    ax.grid(True)
    plt.savefig(os.path.join(path,"".join(row.obsname.unique()[0].split("_")[:-1])+'.png' ))

    

# make plot of only first heads measurement to get map of actual head resisduals

In [ ]:
import mapclassify

In [ ]:
head_diff = []
for n, row in hall.sort_values('nobs', ascending=False).groupby('wellnum'):
    rowind = row['row'].unique()[0]
    column = row['column'].unique()[0]
    
    temp = row.set_index('date').loc[:,'0':].sort_index().head(1)

    #observed - predicted
    temp = row.head(1)['hobs'].values[0] - temp

    # sub = temp.iloc[0,:]
    # temp.iloc[1:] = row['hobs'] - temp.iloc[1:].add(sub)

    temp.index = [row['obsname'].unique()[0]]
    head_diff.append(temp)
    # row.set_index('date').loc[:,'0':].sort_index().add(row.set_index('date').loc[:,'0':].sort_index().iloc[0,:])

head_diff = pd.concat(head_diff)

head_diff = pd.merge(head_diff.T.describe().T, hall.loc[:,:'date'], left_index = True, right_on = 'obsname')
head_diff = pd.merge(head_diff, geom, left_on = ['row', 'column'], right_on = ['row', 'col'])
head_diff = gpd.GeoDataFrame(head_diff, crs = 2226)

q10 = mapclassify.Quantiles(head_diff.loc[:,'mean'].abs(), k=10)
head_diff = head_diff.assign(cl=q10.yb)
head_diff = head_diff.set_geometry(head_diff.geometry.centroid)

nzobs = pst.observation_data.query('weight>0').copy()
nzobs = nzobs.obsnme.apply(lambda x: x.split(":")[-1].upper()).values
nzobs

head_diff =head_diff.loc[head_diff.obsname.isin(nzobs)]

In [ ]:
head_diff.set_geometry(head_diff.geometry.centroid).drop(columns = ['date','totim']).explore()

In [ ]:
head_diff

In [ ]:
hall.query("layer==-3").drop_duplicates(['row','column'])

In [ ]:
os.makedirs(os.path.join(out_folder,'residuals'),exist_ok = True)

In [ ]:
os.listdir(fold)

In [ ]:
pst.parrep(noptmax = 3, parfile = os.path.join)

In [ ]:
from matplotlib.colors import TwoSlopeNorm
# normalize color
vmin, vmax, vcenter = head_diff.loc[:,'mean'].min(), head_diff.loc[:,'mean'].max(), 0
norm = TwoSlopeNorm(vmin=-100, vcenter=vcenter, vmax=100)
# create a normalized colorbar
cmap = 'RdBu'
cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)

## plot all obbs from all layers
fig = plt.figure()
ax = sv_budget.sv_budget.sv_mod_map(simple = True,figsize = (8.5,11))

head_diff.loc[:,'ms'] = head_diff.loc[:,'cl']*5+5
head_diff.plot(ax=ax, column = 'mean', markersize ='ms', cmap=cmap, norm=norm, ec = 'k' )
fig.colorbar(cbar, ax=ax,shrink = .5, extend = 'both')

_ = ph.label_points(ax, head_diff, 'mean',already_str=False, fmt = '+.0f')

plt.savefig(os.path.join(out_folder,'residuals','mean_map_resid.png'))



for layer in range(0,6):
    fig = plt.figure()
    ax = sv_budget.sv_budget.sv_mod_map(simple = True,figsize = (8.5,11))
    sel = head_diff.loc[head_diff.loc[:,f"lay{layer}"]]
    sel.plot(column ='mean',ax=ax, markersize ='ms',  cmap=cmap, norm=norm, ec = 'k' )
    # sel.plot(ax=ax, markersize =sel.loc[:,'ms'], c =sel.loc[:,'mean'], cmap=cmap, norm=norm, ec = 'k' )
    fig.colorbar(cbar, ax=ax,shrink = .5, extend = 'both')
    
    _ = ph.label_points(ax, sel, 'mean',already_str=False, fmt = '+.0f')
    ax.set_title(f'Mean residuals for Absolute Heads, Layer {layer+1}')
    plt.savefig(os.path.join(out_folder,'residuals',f'mean_map_resid_layer_{layer+1}.png'))

In [ ]:
phi_comp = pst.res.copy()

phi_comp.loc[:, 'phi'] = (phi_comp.loc[:,'residual']*phi_comp.loc[:,'weight'])**2
phi_comp.loc[:, 'phi_perc'] = 100*phi_comp.loc[:, 'phi']/pst.phi_components['hds']
print(phi_comp.phi_perc.sum())
phi_comp.sort_values('phi').query("group=='hds'")

In [ ]:
pst.observation_data

In [ ]:
pd.DataFrame().iterrows

In [ ]:
fig, ax = plt.subplots(1, sharex = False)

for c, row in phi_per.iterrows():
    ax.hist(row['0':], bins = 50)


In [ ]:
phi_per.groupby(level = 0)

In [ ]:
phi_per.index


In [ ]:
phi_per.at[3,'mean'] + phi_per.at[3,'standard_deviation'] 

In [ ]:
df.combine_first()

In [ ]:
df = gpd.read_file(r"C:\GIS\shapefiles\wells\RMP_wells\Pet_shp\PET_RMPs.shp")

In [ ]:
df.head()

In [ ]:
stats = conda_scripts.wiski.wiski.get_gw_stations_in_basin(basins = ['Son*'],basic_only = False, add_prmd=True)

In [ ]:
stats = stats.drop_duplicates('station_no')

In [ ]:
import time
start = time.time()
gw = conda_scripts.wiski.wiski.get_gw_stations_wellmeta_in_basin(basins = ['Son*'],wiski_census_info = True, add_prmd=True)
end = time.time()
print(end - start)


In [ ]:
gw = gw.loc[gw.station_no.isin(stats.station_no)]
gwgdf = gpd.GeoDataFrame(gw, geometry = gpd.points_from_xy(gw.station_longitude, gw.station_latitude), crs = 4326)